# Configuration

In [1]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import boto3
from io import StringIO

# Charger les variables d'environnement
load_dotenv()

# Configuration AWS
AWS_REGION = 'eu-west-3'
BUCKET_NAME = 'projey-kayak'
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')

# Initialiser le client S3
s3_client = boto3.client(
    's3',
    region_name=AWS_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

# Obtenir les coordonnées GPS pour chaque ville

In [2]:
BASE_URL = "https://nominatim.openstreetmap.org/search?"

list_cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

headers = {
    'User-Agent': 'kayak_best_destinations/1.0 (YOUR_EMAIL_ADRESS)'
}
# L'API bloque les requêtes si l'on ne donne pas ces infos


In [3]:
#Ici on obtient un dictionnaire avec en clé le nom de la ville et en valeur ses infos
dict_cities = {}

for city in list_cities:
    payload = {'q': city, 'countrycodes': 'fr', 'format': 'json', 'limit': 1}
    response = requests.get(BASE_URL, params=payload, headers=headers)
    dict_cities[city] = response.json()

In [4]:
# Sauvegarder dict_cities sur S3
json_buffer = StringIO()
json.dump(dict_cities, json_buffer, indent=2)

s3_client.put_object(
    Bucket=BUCKET_NAME,
    Key='raw/weather/nominatim_cities.json',
    Body=json_buffer.getvalue()
)

print("✅ dict_cities uploadé sur S3 (raw/weather/nominatim_cities.json)")

✅ dict_cities uploadé sur S3 (raw/weather/nominatim_cities.json)


# Obtenir des informations sur la météo

In [5]:
# Obtenir des informations sur la météo
BASE_URL = 'https://api.openweathermap.org/data/3.0/onecall'
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')

dict_city_meteo = {}

for city in list_cities:
    # Récupérer lat/lon depuis dict_cities
    lon = dict_cities[city][0]['lon']
    lat = dict_cities[city][0]['lat']
    
    payload = {
        'lon': float(lon),
        'lat': float(lat),
        'appid': OPENWEATHER_API_KEY,
        'units': 'metric'
    }
    response = requests.get(BASE_URL, params=payload)
    dict_city_meteo[city] = response.json()

In [6]:
# Sauvegarder dict_city_meteo sur S3
json_buffer = StringIO()
json.dump(dict_city_meteo, json_buffer, indent=2)

s3_client.put_object(
    Bucket=BUCKET_NAME,
    Key='raw/weather/openweather_data.json',
    Body=json_buffer.getvalue()
)

print("✅ dict_city_meteo uploadé sur S3 (raw/weather/openweather_data.json)")

✅ dict_city_meteo uploadé sur S3 (raw/weather/openweather_data.json)
